In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

# defining AC = Aaron Ciminelli. AC will be added to comments added by me 
# AC change animal_shelter and AnimalShelter 
from animalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser" # AC Username required by instructions 
password = "password" # AC password set during setup
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div',
             style={'display': 'none'}),
    #############################################################
    # AC personalizing main header.                             #
    # AC The left is my name with a red highlight               #
    # AC the right is the name of the app with a blue highlight #
    #############################################################
    html.Div([
        html.Div(html.Center(html.B(html.H1('Aaron Ciminelli CS-340 Dashboard',
                                            style={
                                                'color': 'black',
                                                'background-color': '#FF6347',
                                                'text-align': 'center'}))),
                 style={'display': 'inline-block',
                       'padding-left': '20px',
                       'padding-right': '20px'}),
        html.Div(html.Center(html.B(html.H1('Animal Shelter',
                                            style={'color': 'black',
                                                   'background-color': '#87CEEB',
                                                   'text-align': 'top-right'}))),
                 style={'display': 'inline-block',
                       'padding-left': '20px',
                       'padding-right': '20px'}),
    ], style={'text-align': 'center'}),

    #############################################################
    # AC in the table header i removed the _.                   #
    # also the text is capitalized, centered and bold           #
    #                                                           #
    #############################################################
    html.Hr(),
    html.Button('Retrieve All', id='retrieve-all-button'),
    dash_table.DataTable(
        id='datatable-id',
    columns=[
        {"name": i.replace("_", " ").upper(),
         "id": i, "deletable": False,
         "selectable": True} for i in df.columns if i not in ["gender", "age"]
    ],
    data=df.to_dict('records'),
    style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
    },
    style_cell={
        'backgroundColor': 'rgb(248, 248, 248)',
        'textAlign': 'center',

        'whiteSpace': 'normal',
        'textOverflow': 'ellipsis',
        'overflow': 'hidden',
    },
    style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }
    ],
    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    filter_action='native',
    sort_action='native',
    page_action='native',
    selected_rows='[]',
    page_size=20,
),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
    
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'data'),
    [Input('retrieve-all-button', 'n_clicks')]
)
def retrieve_all_data(n_clicks):
    return df.to_dict('records')



@app.callback(
    Output('datatable-id', 'style_data'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{'backgroundColor': '#D2F3FF' if i in selected_rows else ''} for i in range(len(df))]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_row_ids')]
)
def update_map(viewData, selected_row_ids):
    if not selected_row_ids:
        return []
    
    selected_idx = selected_row_ids[0]
    dff = pd.DataFrame.from_dict(viewData)
    selected_row = dff.iloc[selected_idx, :]
    
    location = [selected_row['location_lat'], selected_row['location_long']]
    
    return [dcc.Graph(figure=px.scatter_mapbox(dff, lat="location_lat", lon="location_long",
                                               color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10))]
app

Your Username and Password is: aacuser password
Connection Was Successful
